# Initial Exploration - Optimisation
#### We got a prediction of around 68% last round. Lets see if I can improve it. Also, need to check why cuda isn't working :(
I primarily followed the tutorials here: https://pytorch.org/tutorials/beginner/basics/intro.html

In [17]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [3]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [4]:
training_data = datasets.FashionMNIST(
    root="__local/data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

test_data = datasets.FashionMNIST(
    root="__local/data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [5]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [9]:
model = NeuralNetwork().to(device)
learning_rate = 1e-3
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291070  [   64/60000]
loss: 2.292632  [ 6464/60000]
loss: 2.266061  [12864/60000]
loss: 2.261965  [19264/60000]
loss: 2.230645  [25664/60000]
loss: 2.235674  [32064/60000]
loss: 2.218832  [38464/60000]
loss: 2.208920  [44864/60000]
loss: 2.195314  [51264/60000]
loss: 2.161665  [57664/60000]
Test Error: 
 Accuracy: 51.3%, Avg loss: 2.159155 

Epoch 2
-------------------------------
loss: 2.161879  [   64/60000]
loss: 2.140717  [ 6464/60000]
loss: 2.117254  [12864/60000]
loss: 2.109739  [19264/60000]
loss: 2.036493  [25664/60000]
loss: 2.044000  [32064/60000]
loss: 2.038944  [38464/60000]
loss: 1.989239  [44864/60000]
loss: 1.968734  [51264/60000]
loss: 1.909194  [57664/60000]
Test Error: 
 Accuracy: 57.5%, Avg loss: 1.893681 

Epoch 3
-------------------------------
loss: 1.885677  [   64/60000]
loss: 1.876062  [ 6464/60000]
loss: 1.811948  [12864/60000]
loss: 1.733227  [19264/60000]
loss: 1.767694  [25664/60000]
loss: 1.708964  [32064/600

In [11]:
model = NeuralNetwork().to(device)
learning_rate = 1e-3
epochs = 30
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.311948  [   64/60000]
loss: 2.294716  [ 6464/60000]
loss: 2.286390  [12864/60000]
loss: 2.255832  [19264/60000]
loss: 2.257847  [25664/60000]
loss: 2.229899  [32064/60000]
loss: 2.219047  [38464/60000]
loss: 2.211419  [44864/60000]
loss: 2.175901  [51264/60000]
loss: 2.157623  [57664/60000]
Test Error: 
 Accuracy: 34.4%, Avg loss: 2.150683 

Epoch 2
-------------------------------
loss: 2.146438  [   64/60000]
loss: 2.125218  [ 6464/60000]
loss: 2.085717  [12864/60000]
loss: 2.073854  [19264/60000]
loss: 2.057316  [25664/60000]
loss: 2.020513  [32064/60000]
loss: 1.994612  [38464/60000]
loss: 1.940949  [44864/60000]
loss: 1.884192  [51264/60000]
loss: 1.866383  [57664/60000]
Test Error: 
 Accuracy: 57.5%, Avg loss: 1.869743 

Epoch 3
-------------------------------
loss: 1.881047  [   64/60000]
loss: 1.880343  [ 6464/60000]
loss: 1.779090  [12864/60000]
loss: 1.742642  [19264/60000]
loss: 1.674071  [25664/60000]
loss: 1.577663  [32064/600

In [12]:
model = NeuralNetwork().to(device)
learning_rate = 1e-3
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298174  [   64/60000]
loss: 0.619142  [ 6464/60000]
loss: 0.540211  [12864/60000]
loss: 0.609923  [19264/60000]
loss: 0.470536  [25664/60000]
loss: 0.385651  [32064/60000]
loss: 0.318150  [38464/60000]
loss: 0.394950  [44864/60000]
loss: 0.364814  [51264/60000]
loss: 0.432497  [57664/60000]
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.413418 

Epoch 2
-------------------------------
loss: 0.537303  [   64/60000]
loss: 0.308946  [ 6464/60000]
loss: 0.445579  [12864/60000]
loss: 0.252664  [19264/60000]
loss: 0.327632  [25664/60000]
loss: 0.418841  [32064/60000]
loss: 0.348536  [38464/60000]
loss: 0.410786  [44864/60000]
loss: 0.425308  [51264/60000]
loss: 0.312756  [57664/60000]
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.385581 

Epoch 3
-------------------------------
loss: 0.306918  [   64/60000]
loss: 0.386514  [ 6464/60000]
loss: 0.190221  [12864/60000]
loss: 0.375508  [19264/60000]
loss: 0.243451  [25664/60000]
loss: 0.466014  [32064/600

In [13]:
model = NeuralNetwork().to(device)
learning_rate = 1e-3
epochs = 20
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300718  [   64/60000]
loss: 0.663986  [ 6464/60000]
loss: 0.451158  [12864/60000]
loss: 0.873041  [19264/60000]
loss: 0.497125  [25664/60000]
loss: 0.376960  [32064/60000]
loss: 0.533196  [38464/60000]
loss: 0.387961  [44864/60000]
loss: 0.433978  [51264/60000]
loss: 0.395736  [57664/60000]
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.417189 

Epoch 2
-------------------------------
loss: 0.510163  [   64/60000]
loss: 0.413671  [ 6464/60000]
loss: 0.314601  [12864/60000]
loss: 0.241392  [19264/60000]
loss: 0.241422  [25664/60000]
loss: 0.377913  [32064/60000]
loss: 0.367820  [38464/60000]
loss: 0.497404  [44864/60000]
loss: 0.179050  [51264/60000]
loss: 0.303107  [57664/60000]
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.364598 

Epoch 3
-------------------------------
loss: 0.200794  [   64/60000]
loss: 0.229420  [ 6464/60000]
loss: 0.339456  [12864/60000]
loss: 0.375332  [19264/60000]
loss: 0.269926  [25664/60000]
loss: 0.472511  [32064/600

In [14]:
torch.save(model, '__local/models/1.1-model.pth')

In [16]:
model = torch.load('__local/models/1.1-model.pth')